<a href="https://colab.research.google.com/github/GLasare/Facial-Expression-Recognition-Challenge/blob/main/model_exmperiment_XGBOOST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip walmart-recruiting-store-sales-forecasting.zip -d walmart-recruiting

!unzip walmart-recruiting/features.csv.zip -d walmart-recruiting/
!unzip walmart-recruiting/sampleSubmission.csv.zip -d walmart-recruiting/
!unzip walmart-recruiting/test.csv.zip -d walmart-recruiting/
!unzip walmart-recruiting/train.csv.zip -d walmart-recruiting/

mkdir: cannot create directory ‘/root/.kaggle’: File exists
walmart-recruiting-store-sales-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  walmart-recruiting-store-sales-forecasting.zip
replace walmart-recruiting/features.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: walmart-recruiting/features.csv.zip  
replace walmart-recruiting/sampleSubmission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: walmart-recruiting/sampleSubmission.csv.zip  
replace walmart-recruiting/stores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: walmart-recruiting/stores.csv  
replace walmart-recruiting/test.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: walmart-recruiting/test.csv.zip  
replace walmart-recruiting/train.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: walmart-recruiting/train.csv.zip  
Archive:  walmart-recruiting/features.csv.zip
replace walmart-recruiting/features.c

In [16]:
# should include unzipped data files
!ls -la walmart-recruiting/

total 20924
drwxr-xr-x 2 root root     4096 Jul  2 12:46 .
drwxr-xr-x 1 root root     4096 Jul  2 10:43 ..
-rw-r--r-- 1 root root   592289 Feb  3  2014 features.csv
-rw-r--r-- 1 root root   161700 Dec 11  2019 features.csv.zip
-rw-r--r-- 1 root root  2146739 Feb  3  2014 sampleSubmission.csv
-rw-r--r-- 1 root root   225538 Dec 11  2019 sampleSubmission.csv.zip
-rw-r--r-- 1 root root      532 Dec 11  2019 stores.csv
-rw-r--r-- 1 root root  2598077 Feb  3  2014 test.csv
-rw-r--r-- 1 root root   240940 Dec 11  2019 test.csv.zip
-rw-r--r-- 1 root root 12842546 Feb  3  2014 train.csv
-rw-r--r-- 1 root root  2587161 Dec 11  2019 train.csv.zip


In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split

features = pd.read_csv("walmart-recruiting/features.csv")
train = pd.read_csv("walmart-recruiting/train.csv")
test = pd.read_csv("walmart-recruiting/test.csv")
stores = pd.read_csv("walmart-recruiting/stores.csv")

train_indices, val_indices = train_test_split(
    train.index, test_size=0.2, random_state=42, shuffle=True
)

train_data = train.iloc[train_indices].copy()
val_data = train.iloc[val_indices].copy()

print("Train shape:", train_data.shape)
print("Val shape:", val_data.shape)

Train shape: (337256, 5)
Val shape: (84314, 5)


In [68]:
# Basic stats on training data only
print("\nSales stats (training only):")
print(train_data['Weekly_Sales'].describe())

# Check negatives and zeros
print("\nNegative sales:", (train_data['Weekly_Sales'] < 0).sum())
print("Zero sales:", (train_data['Weekly_Sales'] == 0).sum())

# Store and dept counts
print("\nUnique stores in train:", train_data['Store'].nunique())
print("Unique departments in train:", train_data['Dept'].nunique())

# Date range check
train_data['Date'] = pd.to_datetime(train_data['Date'])
val_data['Date'] = pd.to_datetime(val_data['Date'])
print("\nTrain date range:", train_data['Date'].min(), "to", train_data['Date'].max())
print("Val date range:", val_data['Date'].min(), "to", val_data['Date'].max())

# Sample department analysis
print("\nSample dept sales (from training):")
for dept in [1, 10, 50, 90]:
    dept_sales = train_data[train_data['Dept'] == dept]['Weekly_Sales']
    if len(dept_sales) > 0:
        print(f"Dept {dept}: mean=${dept_sales.mean():.2f}, std=${dept_sales.std():.2f}")

# Check features data
print("\nFeatures shape:", features.shape)
print("Stores shape:", stores.shape)

# Markdown analysis
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
print("\nMarkdown missing percentages:")
for col in markdown_cols:
    print(f"{col}: {features[col].isna().mean() * 100:.1f}%")

# Store types
print("\nStore types:")
print(stores['Type'].value_counts())


Sales stats (training only):
count    337256.000000
mean      15979.221911
std       22679.937123
min       -4988.940000
25%        2080.042500
50%        7607.155000
75%       20236.550000
max      693099.360000
Name: Weekly_Sales, dtype: float64

Negative sales: 1024
Zero sales: 60

Unique stores in train: 45
Unique departments in train: 81

Train date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
Val date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00

Sample dept sales (from training):
Dept 1: mean=$19229.72, std=$15485.10
Dept 10: mean=$18404.36, std=$13015.94
Dept 50: mean=$2644.61, std=$1722.46
Dept 90: mean=$45266.02, std=$32427.35

Features shape: (8190, 12)
Stores shape: (45, 3)

Markdown missing percentages:
MarkDown1: 50.8%
MarkDown2: 64.3%
MarkDown3: 55.9%
MarkDown4: 57.7%
MarkDown5: 50.5%

Store types:
Type
A    22
B    17
C     6
Name: count, dtype: int64


In [25]:
class DataMerger(BaseEstimator, TransformerMixin):
    def __init__(self, features_df: pd.DataFrame, stores_df: pd.DataFrame):
        self.features_df = features_df
        self.stores_df = stores_df

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X = X.merge(self.features_df, on=['Store', 'Date'], how='left')
        X = X.merge(self.stores_df, on='Store', how='left')
        return X

# **Cleaning**

In [26]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class DataCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, remove_negatives=False, cap_outliers=False, outlier_std_threshold=5):
        self.remove_negatives = remove_negatives
        self.cap_outliers = cap_outliers
        self.outlier_std_threshold = outlier_std_threshold
        self.outlier_caps = {}

    def fit(self, X, y=None):
        if self.cap_outliers and 'Weekly_Sales' in X.columns:
            grouped = X.groupby('Dept')['Weekly_Sales']

            for dept, group in grouped:
                mean = group.mean()
                std = group.std()
                upper_cap = mean + self.outlier_std_threshold * std
                lower_cap = mean - self.outlier_std_threshold * std
                self.outlier_caps[dept] = (lower_cap, upper_cap)

        return self

    def transform(self, X):
        X = X.copy()

        if self.remove_negatives and 'Weekly_Sales' in X.columns:
            X = X[X['Weekly_Sales'] >= 0]

        if self.cap_outliers and 'Weekly_Sales' in X.columns:
            for dept, (lower_cap, upper_cap) in self.outlier_caps.items():
                dept_mask = X['Dept'] == dept
                X.loc[dept_mask, 'Weekly_Sales'] = X.loc[dept_mask, 'Weekly_Sales'].clip(
                    lower=lower_cap, upper=upper_cap
                )

        X = X.drop_duplicates(subset=['Store', 'Dept', 'Date'], keep='first')

        return X

In [27]:
class MissingValueHandler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.fill_values = {}

    def fit(self, X, y=None):
        if 'CPI' in X.columns:
            self.fill_values['CPI'] = X.groupby('Store')['CPI'].transform(
                lambda x: x.fillna(method='ffill').fillna(method='bfill')
            )

        if 'Unemployment' in X.columns:
            self.fill_values['Unemployment'] = X.groupby('Store')['Unemployment'].transform(
                lambda x: x.fillna(method='ffill').fillna(method='bfill')
            )

        return self

    def transform(self, X):
        X = X.copy()

        if 'CPI' in X.columns and X['CPI'].isna().any():
            grouped_cpi = X.groupby('Store')['CPI'].transform(
                lambda x: x.fillna(method='ffill').fillna(method='bfill')
            )
            X['CPI'] = grouped_cpi.fillna(X['CPI'].mean())

        if 'Unemployment' in X.columns and X['Unemployment'].isna().any():
            grouped_unemp = X.groupby('Store')['Unemployment'].transform(
                lambda x: x.fillna(method='ffill').fillna(method='bfill')
            )
            X['Unemployment'] = grouped_unemp.fillna(X['Unemployment'].mean())

        return X

# Feature Engineering

In [28]:
class TemporalFeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['Date'] = pd.to_datetime(X['Date'])

        X['Week'] = X['Date'].dt.isocalendar().week
        X['Month'] = X['Date'].dt.month
        X['Quarter'] = X['Date'].dt.quarter
        X['Year'] = X['Date'].dt.year
        X['DayOfYear'] = X['Date'].dt.dayofyear
        X['WeekOfMonth'] = (X['Date'].dt.day - 1) // 7 + 1

        X['Month_Sin'] = np.sin(2 * np.pi * X['Month'] / 12)
        X['Month_Cos'] = np.cos(2 * np.pi * X['Month'] / 12)
        X['Week_Sin'] = np.sin(2 * np.pi * X['Week'] / 52)
        X['Week_Cos'] = np.cos(2 * np.pi * X['Week'] / 52)

        return X

In [29]:
class LagFeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self, lag_weeks=[1, 2, 3, 4, 52],
                 rolling_windows=[4, 8, 52]):
        self.lag_weeks = lag_weeks
        self.rolling_windows = rolling_windows

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X = X.sort_values(['Store', 'Dept', 'Date'])

        grouped = X.groupby(['Store', 'Dept'])

        for lag in self.lag_weeks:
            X[f'Sales_Lag_{lag}'] = grouped['Weekly_Sales'].shift(lag)

        for window in self.rolling_windows:
            X[f'Sales_MA_{window}'] = grouped['Weekly_Sales'].shift(1).rolling(window, min_periods=1).mean()
            X[f'Sales_STD_{window}'] = grouped['Weekly_Sales'].shift(1).rolling(window, min_periods=1).std()

        X['Sales_Lag_52_YoY_Change'] = (X['Weekly_Sales'] - X['Sales_Lag_52']) / X['Sales_Lag_52']

        return X

In [30]:
class HolidayProximityEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.holidays = {
            'SuperBowl': ['2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08'],
            'LaborDay': ['2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06'],
            'Thanksgiving': ['2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29'],
            'Christmas': ['2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27']
        }

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['Date_temp'] = pd.to_datetime(X['Date'])

        for holiday_name, dates in self.holidays.items():
            holiday_dates = pd.to_datetime(dates)

            X[f'{holiday_name}_Weeks_Until'] = X['Date_temp'].apply(
                lambda x: min([(d - x).days // 7 for d in holiday_dates if d >= x], default=999)
            )
            X[f'{holiday_name}_Weeks_Since'] = X['Date_temp'].apply(
                lambda x: min([(x - d).days // 7 for d in holiday_dates if d <= x], default=999)
            )

            X[f'{holiday_name}_Within_2_Weeks'] = (
                (X[f'{holiday_name}_Weeks_Until'] <= 2) |
                (X[f'{holiday_name}_Weeks_Since'] <= 2)
            ).astype(int)

        X.drop('Date_temp', axis=1, inplace=True)
        return X

In [31]:
class DepartmentStoreInteractionEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.store_avg_sales = X.groupby('Store')['Weekly_Sales'].mean()
        self.dept_avg_sales = X.groupby('Dept')['Weekly_Sales'].mean()
        self.store_dept_avg = X.groupby(['Store', 'Dept'])['Weekly_Sales'].mean()
        return self

    def transform(self, X):
        X = X.copy()

        X['Store_Avg_Sales'] = X['Store'].map(self.store_avg_sales)
        X['Dept_Avg_Sales'] = X['Dept'].map(self.dept_avg_sales)
        X['Store_Dept_Avg_Sales'] = X.set_index(['Store', 'Dept']).index.map(self.store_dept_avg)

        X['Dept_Sales_Ratio_To_Store'] = X['Store_Dept_Avg_Sales'] / X['Store_Avg_Sales']
        X['Store_Size_Dept_Interaction'] = X['Size'] * X['Dept_Sales_Ratio_To_Store']

        return X

# Feature Selection

In [32]:
class CorrelationFilter(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.8):
        self.threshold = threshold
        self.to_drop = None

    def fit(self, X, y=None):
        numeric_features = X.select_dtypes(include=[np.number]).columns
        corr_matrix = X[numeric_features].corr().abs()

        upper_triangle = corr_matrix.where(
            np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
        )

        self.to_drop = set()
        for column in upper_triangle.columns:
            if column in self.to_drop:
                continue
            correlated_features = list(upper_triangle.index[upper_triangle[column] > self.threshold])
            self.to_drop.update(correlated_features)

        return self

    def transform(self, X):
        return X.drop(columns=self.to_drop)

In [33]:
class ShapFeatureSelector(BaseEstimator, TransformerMixin):
   def __init__(self, importance_threshold=0.95, sample_size=1000):
       self.importance_threshold = importance_threshold
       self.sample_size = sample_size
       self.selected_features = None
       self.feature_importance_df = None

   def fit(self, X, y):
       import xgboost as xgb
       import shap

       excluded_cols = ['Weekly_Sales', 'Date', 'Store', 'Dept']
       feature_cols = [col for col in X.columns if col not in excluded_cols]

       X_sample = X[feature_cols].sample(n=min(self.sample_size, len(X)), random_state=42)
       y_sample = y.loc[X_sample.index]

       model = xgb.XGBRegressor(
           n_estimators=100,
           max_depth=6,
           learning_rate=0.1,
           random_state=42
       )
       model.fit(X_sample, y_sample)

       explainer = shap.TreeExplainer(model)
       shap_values = explainer.shap_values(X_sample)

       mean_abs_shap = np.abs(shap_values).mean(axis=0)

       self.feature_importance_df = pd.DataFrame({
           'feature': feature_cols,
           'importance': mean_abs_shap
       }).sort_values('importance', ascending=False)

       self.feature_importance_df['importance_normalized'] = (
           self.feature_importance_df['importance'] /
           self.feature_importance_df['importance'].sum()
       )
       self.feature_importance_df['cumulative_importance'] = (
           self.feature_importance_df['importance_normalized'].cumsum()
       )

       n_features_to_keep = (
           self.feature_importance_df['cumulative_importance'] <= self.importance_threshold
       ).sum() + 1

       self.selected_features = list(
           self.feature_importance_df.head(n_features_to_keep)['feature']
       )
       self.selected_features.extend(excluded_cols)

       return self

   def transform(self, X):
       return X[self.selected_features]

In [34]:
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def create_xgboost_preprocessing_pipeline(train_df, features_df, stores_df):

   pipeline = Pipeline([
       ('data_merger', DataMerger(features_df=features_df, stores_df=stores_df)),
       ('data_cleaner', DataCleaner(remove_negatives=False, cap_outliers=False)),
       ('missing_handler', MissingValueHandler()),
       ('temporal_features', TemporalFeatureEngineer()),
       ('lag_features', LagFeatureEngineer(lag_weeks=[1, 2, 3, 4, 52],
                                          rolling_windows=[4, 8, 52])),
       ('holiday_features', HolidayProximityEngineer()),
       ('dept_store_features', DepartmentStoreInteractionEngineer()),
       ('correlation_filter', CorrelationFilter(threshold=0.95)),
       ('shap_selector', ShapFeatureSelector(importance_threshold=0.95, sample_size=5000))
   ])

   return pipeline

def prepare_data_for_xgboost(train_df, features_df, stores_df):

   pipeline = create_xgboost_preprocessing_pipeline(train_df, features_df, stores_df)

   y = train_df['Weekly_Sales'].copy()

   X_transformed = pipeline.fit_transform(train_df, y)

   feature_cols = [col for col in X_transformed.columns
                  if col not in ['Weekly_Sales', 'Date', 'Store', 'Dept']]

   X_final = X_transformed[feature_cols]

   return X_final, y, pipeline, feature_cols

def transform_test_data(test_df, fitted_pipeline, feature_cols):

   X_test_transformed = fitted_pipeline.transform(test_df)
   X_test_final = X_test_transformed[feature_cols]

   return X_test_final

In [70]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# Use the same train/val split from analysis
train_df = train.iloc[train_indices].copy()
val_df = train.iloc[val_indices].copy()

# Fit pipeline on training data only
X_train_processed, y_train_processed, fitted_pipeline, feature_cols = prepare_data_for_xgboost(
    train_df, features, stores
)

# Transform validation data
X_val_processed = transform_test_data(val_df, fitted_pipeline, feature_cols)
y_val_processed = val_df['Weekly_Sales']

print("Processed train shape:", X_train_processed.shape)
print("Processed val shape:", X_val_processed.shape)
print("Number of features:", len(feature_cols))

# Train model
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

print("\nTraining XGBoost...")
xgb_model.fit(X_train_processed, y_train_processed)

# Predictions
y_pred = xgb_model.predict(X_val_processed)

# Get holiday info for WMAE calculation
val_with_holidays = val_df.merge(
    features[['Store', 'Date', 'IsHoliday']],
    on=['Store', 'Date'],
    how='left'
)

weights = np.where(val_with_holidays['IsHoliday'], 5, 1)
wmae = np.sum(weights * np.abs(y_val_processed - y_pred)) / np.sum(weights)

print("\nResults:")
print("WMAE:", wmae)
print("MAE:", mean_absolute_error(y_val_processed, y_pred))
print("Holiday weeks in val:", (weights == 5).sum())
print("Non-holiday weeks in val:", (weights == 1).sum())

Train shape: (335761, 5)
Val shape: (85809, 5)
Train date range: 2010-02-05 00:00:00 to 2012-04-06 00:00:00
Val date range: 2012-04-13 00:00:00 to 2012-10-26 00:00:00


ValueError: You are trying to merge on datetime64[ns] and object columns for key 'Date'. If you wish to proceed you should use pd.concat